In [1]:
from downloader import download_datasets
from dataset_preparation import prepare_datasets

In [2]:
dataset_list = ["guzman_2015", "maalej_2016", "williams_2017"]
training_type = "requirements_relevance"
model_name = "distilbert-base-uncased"

In [3]:
download_datasets(dataset_list, training_type)

In [3]:
train_dataset, val_dataset, test_dataset, label_order = prepare_datasets(dataset_list, training_type, model_name)

In [4]:
label_order

Index(['irrelevant', 'relevant', 'text'], dtype='object')

In [ ]:
from transformers import AutoModelForSequenceClassification, Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
)

model = AutoModelForSequenceClassification.from_pretrained(model_name)

trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset             # evaluation dataset
)

trainer.train()